In [428]:
import pathlib
import json

from attrs import define, field, evolve
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

In [429]:
TEST_DATA = pathlib.Path('/home/michael_olaya_student_uml_edu/repos/odb-extractor/tests/raw_odbex/extracted_rimr_carbon_1fib_vf_0p55_RI_fbc_16cpu_22.json')

In [430]:
@define
class FieldData:
    step: str
    region: str
    increments: dict[int, float]
    field: str
    data: np.ndarray
    components: list[str]

    @classmethod
    def from_dict(cls, step: str, region: str, increments: dict[int, float], field: str, field_data_dict: dict[str, list]):
        data = cls._load_field_data(field_data_dict['data'])
        components = field_data_dict['components']
        return cls(step, region, field, increments, data, components)

    @staticmethod
    def _load_field_data(field_data: dict) -> np.ndarray:
        return np.array([d for d in field_data])

@define
class RegionData:
    region: str
    field_data: dict[str, FieldData] = field(factory=dict)

    def volume_average_field(self, field: str) -> np.ndarray:
        if 'IVOL' not in self.field_data.keys():
            print('error: IVOL was not found in the available field data for the region')
            print('ensure IVOL is requested for simulation field outputs and in the odbex config file when extracting data')
            return
        field_data = self.field_data[field].data
        ipvol = self.field_data['IVOL'].data
        vol_averaged = np.array([np.sum(fd*ipv, axis=0)/np.sum(ipv) for fd, ipv in zip(field_data, ipvol)])
        return evolve(self.field_data[field], data=vol_averaged)

@define
class StepData:
    step: str
    region_data: dict[str, RegionData] = field(factory=dict)
    
@define
class SimulationData:
    step_data: dict[str, StepData] = field(factory=dict)

    @classmethod
    def from_extracted(cls, data_filepath: pathlib.Path):
        # Initialize
        cls_ = cls()

        # Load raw data from json
        with open(data_filepath, 'r') as f:
            raw_data = json.load(f)

        # Update dictionary of step data with StepData/RegionData/FieldData objects
        for step, step_data in raw_data.items():
            sd = StepData(step)
            increments = step_data['increments']
            for region, field_data_dicts in step_data['field_data'].items():
                sd.region_data.update({region: RegionData(
                    region=region,
                    field_data={field: FieldData.from_dict(
                        step, region, increments, field, fdd
                    ) for field, fdd in field_data_dicts.items() if not field.lower() in ['elems', 'ips', 'nodes']}
                )})
            cls_.step_data.update({step: sd})
        return cls_
    
    def get_region_data(self, step: str, region: str) -> RegionData:
        return self.step_data[step].region_data[region]

    def get_field_data(self, step: str, region: str, field: str) -> FieldData:
        return self.get_region_data(step, region).field_data[field]

In [431]:
def load_field_data(region_data: dict, name: str) -> list[np.ndarray]:
    return np.array([d for d in region_data[name]['data']])

def volume_average_field(field_data: list[np.ndarray], ipvol: list[np.ndarray]) -> np.ndarray:
    return np.array([np.sum(fd*ipv, axis=0)/np.sum(ipv) for fd, ipv in zip(field_data, ipvol)])

In [434]:
simdata = SimulationData.from_extracted(TEST_DATA)
stress = simdata.get_region_data('pull-step', 'SET-ALLELEMENTS').volume_average_field('S')
strain = simdata.get_region_data('pull-step', 'SET-ALLELEMENTS').volume_average_field('E')

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.21584136e-07,  1.99066891e+01, -2.31749442e-09,
         4.44546151e-10,  2.07722246e-11, -1.28863115e-03],
       [-7.48305691e-03,  3.86786686e+01, -7.47573531e-03,
         8.60983456e-10,  4.11292790e-11, -1.49741180e-03],
       [ 7.52731454e-10,  1.04240778e-02, -5.67714543e-10,
        -4.17199498e-13,  1.86530817e-13,  1.03328866e-03],
       [ 9.90302695e-10,  1.38987704e-02,  1.55049244e-10,
        -5.56267663e-13,  2.48709491e-13,  1.37771835e-03],
       [ 9.73736089e-10,  1.73734632e-02, -3.74146522e-10,
        -6.95332745e-13,  3.10887787e-13,  1.72214796e-03]])

In [433]:
fig, ax = plt.subplots()
ax.plot(strain.data[:, 1], stress.data[:, 1])